In [1]:
logs = sc.textFile("hdfs://l41-srv-mcdh01.b.internal/datasets/magichour/tbird.log.gz").repartition(30)
simpleTransformations = sc.textFile("hdfs://l41-srv-mcdh01.b.internal/magichour/simpleTrans")

In [2]:
#logs.count()
#simpleTransformations.count()

In [3]:
from collections import namedtuple
import re

TransformLine = namedtuple('TransformLine',['id','type','NAME','transform','compiled'])
LogLine = namedtuple('LogLine', ['ts','msg','processed','dictionary','supportId'])

def get_Transforms(line):
    if line.lstrip()[0] != '#':
        l = line.lstrip().rstrip().split(',', 3)  #id,type,name,transform
        return TransformLine(int(l[0]),l[1],l[2],l[3],re.compile(l[3]))
    else:
        return TransformLine('COMMENT','COMMENT','COMMENT','COMMENT','COMMENT')

logTransforms = simpleTransformations.map(get_Transforms).cache()
#logTransformList = sc.broadcast(list(simpleTransformations))



In [4]:
# logTransforms.take(5)

In [5]:
a = logTransforms.collect()
lTrans = list()

for l in a:
    if l.id != 'COMMENT':
        lTrans.append(l)
        
logTrans= sc.broadcast(lTrans)

In [6]:
potato = tsLine.map(makeTransformedLine)

NameError: name 'tsLine' is not defined

In [ ]:
potato.take(100)

In [ ]:
potato.count()

In [ ]:
from collections import namedtuple
import re

TransformLine = namedtuple('TransformLine',
                           ['id', 'type', 'NAME', 'transform', 'compiled'])

LogLine = namedtuple('LogLine', ['ts', 'msg',
                                 'processed', 'dictionary', 'supportId'])


def rdd_TransformLine(line):
    '''
    process transformations into RDD format

    Args:
        line(string): line from the transform defintion file.
                      lines beginning with # are considered comments
                      and will need to be removed
    Returns:
        retval(TransformLine): namedTuple representation of the tasking
    '''

    if line.lstrip()[0] != '#':
        # id,type,name,transform
        l = line.lstrip().rstrip().split(',', 3)
        return TransformLine(int(l[0]), l[1], l[2], l[3], re.compile(l[3]))
    else:
        return TransformLine('COMMENT',
                             'COMMENT',
                             'COMMENT',
                             'COMMENT',
                             'COMMENT')


def rdd_LogLine(line):
    '''
    process a log line into a RDD

    Args:
        line(string): string from the logline

    Returns:
        retval(LogLine): fills in the first two portions of the LogLine
                         namedtuple
    '''

    # depends on tbird log structure
    l = line.strip().rstrip().split(' ', 3)
    return LogLine(float(l[2]), l[3], None, None, None)


def lineRegexReplacement(line):
    '''
    apply a list of regex replacements to a line, make note of
    all the remplacements peformed in a dictionary(list)

    Args:
        line(LogLine): logline to work on

    Globals:
        transforms(RDD(TransformLine)): replacemnts to make with

    Returns:
        retval(LogLine): logline with the processed, and dictionary portions
                         filled in
    '''
    global logTrans
    text = line.msg.strip()
    replaceDict = dict()

    # logTrans is broadcast
    for t in logTrans.value:
        if t.type == 'REPLACE':
            replaceList = t.compiled.findall(text)
            if replaceList:
                replaceDict[t.NAME] = replaceList
            text = t.compiled.sub(t.NAME, text, 0)

        if t.type == 'REPLACELIST':
            print 'REPLACELIST not implemented yet'

    processed = ' '.join(text.split())
    retVal = LogLine(line.ts, line.msg.lstrip().rstrip(),
                     processed.lstrip().rstrip(), replaceDict, None)

    return retVal


def readTransforms(transFile):

    # map the transFile
    simpleTransformations = sc.textFile(transFile)

    # parse loglines
    return simpleTransformations.map(rdd_TransformLine).cache()

   

def doPreProcess(logFile, transFile, partitions):
    '''
        take a series of loglines and pre-process the lines
        replace ipaddresses, directories, urls, etc with constants
        keep a dictionary of the replacements done to the line

        Args:
            logFile(string): location of the log data in HDFS
            transFile(string): location of the replacement regex in HDFS
            partitions(int): number of partitions to apply to the logFile

        Returns:
            retval(RDD(LogLines)): preprocessed log lines ready for next
                                   stage of processing
   '''

    # read the logs
    logs = sc.textFile(logFile).repartition(partitions)

    # read the transforms, removing comments
    logTransforms = readTransforms(transFile)

    trans = logTransforms.collect()
    lTrans = list()

    for t in trans:
        if t.id != 'COMMENT':
            lTrans.append(t)

    logTrans = sc.broadcast(lTrans)

    
    tsLine = logs.map(rdd_LogLine)
    return tsLine.map(lineRegexReplacement)







In [ ]:
logs = 'hdfs://l41-srv-mcdh01.b.internal/datasets/magichour/tbird.log.gz'
logs = 'hdfs://l41-srv-mcdh01.b.internal/user/dgrossman/tbird.log.10000.gz'
trans = 'hdfs://l41-srv-mcdh01.b.internal/magichour/simpleTrans'


In [30]:
out = doPreProcess(logs,trans,1)

In [31]:
out.take(100)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 4 times, most recent failure: Lost task 0.3 in stage 15.0 (TID 35, 10.161.0.19): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cdh_data/mesos/slaves/20150827-215802-671129866-5050-2150-S19/frameworks/20150827-215802-671129866-5050-2150-0255/executors/13/runs/1f9864d2-de6b-4432-8b3a-7a70e326bce4/spark-1.6.0-bin-hadoop2.4/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/cdh_data/mesos/slaves/20150827-215802-671129866-5050-2150-S19/frameworks/20150827-215802-671129866-5050-2150-0255/executors/13/runs/1f9864d2-de6b-4432-8b3a-7a70e326bce4/spark-1.6.0-bin-hadoop2.4/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/cdh_data/me